In [100]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
 


In [108]:
# this is just to visualize the posts
df = pd.read_csv('./posts.csv')
df.drop(['Author', 'Index','Category', 'Age range'], axis=1, inplace=True)
df.head()

,Title,Body
0,The Boy Who Cried Wolf!,"A shepherd-boy, who watched a flock of sheep n..."
1,The Girl who tried to fly,"Once upon a time, there was a girl named Lily...."
2,Monster in the Jungle,"Once upon a time, a wise lion lived in the jun..."
3,THE ANT AND THE GRASSHOPPER,The importance of hard work and planning for t...
4,A friend in need is a friend indeed,"Once upon a time, there lived a lion in a fore..."


In [110]:
def read_article(file_name):
    file = open("posts.csv", "r")
    filedata = file.readlines()
    article = filedata[20].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix




In [113]:
def generate_summary(file_name, top_n=10):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Read text and split it
    sentences =  read_article("posts.csv")

    # Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # output the summarize text
    print("Summarize Text: \n", ". ".join(summarize_text))

# let's begin
#generate_summary( "posts.csv", 6)


In [114]:
generate_summary("posts.csv", 7)


20,Jolly Robin Learns To Fly,null,0 to 5,"Jolly Robin had just gulped down the fat worm that his father had used to coax him out of the nest
And now he clung desperately to the limb
With no food in sight he had plenty of time to look around
And he was very alarmed! The day would not be over before he had another great fright
Just then, Jolly lost his grip on the limb and tumbled out of the apple tree and fell squawking and fluttering to the ground
Luckily his mother went to Jolly immediately and told him, �Do not be afraid
Nothing will hurt you,� she said, �if you�ll only keep still
But if you squawk like that, the cat will find you.� His mother�s words frightened Jolly very much
They scared him so that he stopped making the noise
And that was how he learned the lesson never to talk when he was on the ground near a house where a cat might live
�Now,� said mother, as soon as Jolly was still, �I�ll teach you a new game
Just watch me!� She spread her wings, she flapped them and sprang int